### Sustainable and Entrepreneurial Finance
### Assignment 1 - Portfolio allocation
#### Group 8 - Energy Firms With Available Scope 1 to 3 emissions (TRUCOST)
Useful imports:

In [3]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import random

pd.options.plotting.backend = "plotly"

#### 0 - Importing and preparing datasets for calculation.
Importing the files and creating pandas data frames

In [64]:
# Setting path names
path_gics = './Data_Excel/Trucost_CO2emissions/GICS_map 2018.xlsx'
path_sector = './Data_Excel/Trucost_CO2emissions/sector.xlsx'
path_returns = './Data_Excel/MSCI_ESGscores/Returns/monthlyreturns.xlsx'

# Reading excel files and creating pandas data frames
df_gics = pd.read_excel(path_gics)
df_sector = pd.read_excel(path_sector)
df_returns = pd.read_excel(path_returns)

Renaming the index for returns from '`Unnamed: 0`' to '`Date`'.

In [65]:
# Renaming index data column

df_returns.rename(columns = {'Unnamed: 0':'Date'}, inplace = True)
#df_returns.set_index("Date", inplace=True)

Getting the ISIN codes for the energy companies.

In [66]:
industry_code = 1010.0 # based on Global Industry Classification Standard GICS
df_energy = df_sector.loc[df_sector['GICSIG'] == industry_code]
energy_isin = df_energy['ISIN'].values.tolist()

Getting the returns for the companies matching the ISIN codes in `energy_isin`.

In [67]:
# List of all ISIN codes to iterate through
return_cols = df_returns.columns.values.tolist()
display(len(return_cols))
display(len(return_cols) == df_returns.shape[1])

# Creating a list with all the ISIN Energy codes that the returns.xlsx datasheet contains
both = []
for c in return_cols:
  if c in energy_isin:
    both.append(c)

5141

True

Checking the shape and general characteristics of our new list.

In [68]:
display(return_cols[:4])
display(energy_isin[:4])
display(both[:4])
display(len(return_cols))
display(len(energy_isin))
display(len(both))

['Date', 'AEA000201011', 'AEA001501013', 'AEA002001013']

['AED000701014', 'AN8068571086', 'ARP9897X1319', 'ARPERE010103']

['AN8068571086', 'AU000000AOE6', 'AU000000ERA9', 'AU000000ORG5']

5141

773

223

Inserting the Date column. 

In [69]:
if 'Date' not in energy_isin:
  energy_isin.insert(0, 'Date')
else:
  pass

energy_isin[:4]

['Date', 'AED000701014', 'AN8068571086', 'ARP9897X1319']

Putting together the return data for the energy companies.

In [70]:
nrg_returns = df_returns[df_returns.columns.intersection(energy_isin)]
display(nrg_returns)

# Checking the datatypes.
display(nrg_returns.dtypes.unique())

,Date,AN8068571086,AU000000AOE6,AU000000ERA9,AU000000ORG5,AU000000PDN8,AU000000WHC8,AU000000WOR2,BE0003816338,BMG2108M2182,...,US8475601097,US86764P1093,RU000A0B6NK6,US87612G1013,US9026531049,US91913Y1001,US9694571004,US98385X1063,ARP9897X1319,ZAE000006896
0,2002-01-31,0.115662,0.152793,0.005659,-0.031864,-0.091918,NaN,NaN,NaN,-0.039490,...,NaN,0.021051,NaN,NaN,0.002783,0.040387,-0.027370,0.078207,0.455740,0.049202
1,2002-02-28,0.004551,-0.076059,0.075563,0.034254,0.947644,NaN,NaN,NaN,0.026525,...,NaN,0.032410,NaN,NaN,0.002702,0.207248,-0.255487,-0.050850,-0.100205,0.071560
2,2002-03-31,0.069747,0.250782,0.032138,0.046824,-0.051075,NaN,NaN,NaN,-0.040052,...,NaN,0.018067,NaN,NaN,0.012703,-0.055308,-0.157891,0.131839,-0.102242,0.048680
3,2002-04-30,-0.011214,0.161899,0.026068,0.072392,0.106704,NaN,NaN,NaN,-0.056528,...,NaN,0.036674,NaN,NaN,0.108710,0.130936,0.497679,0.096313,0.023116,0.123294
4,2002-05-31,-0.033508,-0.076482,-0.084686,0.054394,-0.104949,NaN,NaN,NaN,0.000000,...,NaN,-0.128155,NaN,NaN,0.063592,-0.111495,-0.186524,0.008259,-0.015311,-0.022361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,2020-08-31,0.056431,0.025929,0.059829,-0.030567,0.221436,-0.051895,-0.087717,0.192621,0.001988,...,0.0,0.000000,-0.040000,-0.015564,-0.087904,-0.027463,0.048380,0.000000,0.286645,0.082327
224,2020-09-30,0.001871,0.038956,0.102151,0.026130,0.412873,-0.301476,0.253637,-0.037486,-0.034939,...,0.0,0.000000,-0.020833,-0.096853,-0.014810,-0.026711,0.062879,0.000000,-0.143275,0.036017
225,2020-10-31,-0.194313,-0.026587,-0.112195,-0.210594,-0.255603,0.193846,-0.000491,-0.039670,-0.147194,...,0.0,0.000000,-0.021277,-0.199779,-0.023121,-0.232324,-0.086246,0.000000,-0.258657,-0.096972
226,2020-11-30,0.037976,-0.019773,-0.052198,-0.102304,-0.095238,-0.081615,-0.061808,-0.135534,-0.106459,...,0.0,0.000000,-0.108696,0.190719,-0.020107,-0.009433,-0.000533,0.000000,-0.083209,-0.291075


array([dtype('<M8[ns]'), dtype('float64')], dtype=object)

The dataset looks good. We have only float64 values which is as expected and good. Additionally we see that we have 223 columns which is the same as the length of the ISIN list created in the codeblock above.

Now we can filter on the targeted dates which is from 01.01.2005 to 31.12.2020.

In [71]:
start_date = '2005-01-01'
end_date = '2020-12-31'

# Greater than or equal to the start date and smaller than or equal the end date
mask = (nrg_returns['Date'] >= start_date) & (nrg_returns['Date'] <= end_date)

nrg_returns = nrg_returns.loc[mask]
nrg_returns

,Date,AN8068571086,AU000000AOE6,AU000000ERA9,AU000000ORG5,AU000000PDN8,AU000000WHC8,AU000000WOR2,BE0003816338,BMG2108M2182,...,US8475601097,US86764P1093,RU000A0B6NK6,US87612G1013,US9026531049,US91913Y1001,US9694571004,US98385X1063,ARP9897X1319,ZAE000006896
36,2005-01-31,0.012603,-0.087586,0.032408,0.001413,-0.058855,NaN,0.053326,-0.221404,-0.000471,...,NaN,-0.031204,NaN,NaN,0.047824,-0.035393,-0.046450,-0.060108,0.028704,0.088215
37,2005-02-28,0.063502,-0.006316,0.509758,0.008180,0.818187,NaN,0.037500,0.031757,-0.012241,...,NaN,0.161942,NaN,NaN,-0.069058,0.322805,0.092083,0.108183,0.094693,-0.087507
38,2005-03-31,0.093039,0.131431,0.188845,0.023138,0.373450,NaN,0.130329,0.230611,0.163489,...,NaN,0.080660,NaN,NaN,-0.008869,0.196423,0.077830,0.183483,0.258207,0.231228
39,2005-04-30,-0.038401,-0.053780,-0.130408,0.031135,-0.114356,NaN,0.049456,0.038771,0.140516,...,NaN,0.107051,NaN,NaN,-0.067113,0.142520,0.045476,0.067322,-0.103228,0.002223
40,2005-05-31,-0.027085,-0.122518,-0.048968,-0.014060,-0.006077,NaN,0.004279,0.031982,0.000359,...,NaN,-0.066758,NaN,NaN,0.083772,-0.118698,-0.084998,-0.109858,-0.011597,-0.042448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,2020-08-31,0.056431,0.025929,0.059829,-0.030567,0.221436,-0.051895,-0.087717,0.192621,0.001988,...,0.0,0.000000,-0.040000,-0.015564,-0.087904,-0.027463,0.048380,0.000000,0.286645,0.082327
224,2020-09-30,0.001871,0.038956,0.102151,0.026130,0.412873,-0.301476,0.253637,-0.037486,-0.034939,...,0.0,0.000000,-0.020833,-0.096853,-0.014810,-0.026711,0.062879,0.000000,-0.143275,0.036017
225,2020-10-31,-0.194313,-0.026587,-0.112195,-0.210594,-0.255603,0.193846,-0.000491,-0.039670,-0.147194,...,0.0,0.000000,-0.021277,-0.199779,-0.023121,-0.232324,-0.086246,0.000000,-0.258657,-0.096972
226,2020-11-30,0.037976,-0.019773,-0.052198,-0.102304,-0.095238,-0.081615,-0.061808,-0.135534,-0.106459,...,0.0,0.000000,-0.108696,0.190719,-0.020107,-0.009433,-0.000533,0.000000,-0.083209,-0.291075


#### I - Annual average return and annualized volatility for all individual assets over the period 2005-2020. Correlation between individual average returns and volatility.

#### II - Equally-weighted and value-weighted portfolio with monthly rebalancing over the period 2005-2020. Report the following statistics for both portfolios: annualized average return, annualized volatility, minimum return, maximum return, and Sharpe ratio. Plot the time series of return for both portfolios

#### III - For this question, limit your set of firms to 100 randomly selected firms. Pay a particular attention to the construction of the covariance matrix. Build an optimal portfolio with minimum variance with monthly rebalancing over the period 2005-2020. Report the following statistics: annualized average return, annualized volatility, minimum return, maximum return, and Sharpe ratio. Comment on the reported statistics in comparison with the equally-weighted and value-weighted portfolio

#### IV - For this question, keep the same randomly selected firms from the previous point. Build an optimal portfolios with various target portfolio returns (e.g., from 2% to 16% with 2% increments). Plot the efficient frontier as well as the individual assets. Which portfolio is the most efficient in terms of Sharpe ratio?

#### V - Choose an appropriate benchmark, which corresponds to the region of your dataset. Compare the performance of your portfolios (equally-weighted, value-weighted, and minimum variance) with the benchmark. Comment on the differences.

#### VI - Compute and comment on the simple correlation between returns, volatility, size.

#### VII - For this question, take the same 100 selected firms. You now create a minimum variance portfolio with monthly rebalancing with an additional constraint: you exclude the smallest firms (bottom tercile of the distribution of the firms’ market capitalization in month t − 1). Report summary statistics on the performance of this portfolio and comment on the differences with the minimum variance from point 3. 